In [29]:
import numpy as np
import time


np.random.seed(3)
batch_size = 10
n_node = 20
n_epoch=2 
n_batch_per_epoch = 100
tasks = np.random.uniform(size=(n_epoch, n_batch_per_epoch,batch_size, n_node, 2)).astype(np.float32)

tasks.shape

(2, 100, 10, 20, 2)

In [30]:
import math
def calculate_angle(point1, point2):
    # 计算两点之间的角度
    delta_x = point2[0] - point1[0]
    delta_y = point2[1] - point1[1]
    angle = math.atan2(delta_y, delta_x) * 180 / math.pi
    return angle

def cal_angle_and_mask_matrix(tasks):
    '''Calculate angle between tasks and mask matrix.
    Args:
        tasks (np.array): (batch_size, n_node, 2)
    Returns:
        angle (np.array): (batch_size, n_node, n_node)
        '''
    n_epoch, n_batch_per_epoch,batch_size, n_node, _ = tasks.shape
    angle = np.zeros((n_epoch, n_batch_per_epoch,batch_size, n_node, n_node))
    mask_matrix = np.zeros((n_epoch, n_batch_per_epoch,batch_size, n_node, n_node))
    for m in range(n_epoch):
        for n in range(n_batch_per_epoch):
            for i in range(batch_size):
                for j in range(n_node):
                    for k in range(n_node):
                        angle[m,n,i,j,k] = calculate_angle(tasks[m,n,i,j], tasks[m,n,i,k])
                        
                        if -30 < angle[m,n,i,j,k] < 30:
                            mask_matrix[m,n,i,j,k] = 1
    
    return angle, mask_matrix



In [31]:
import numpy as np


def calculate_angle_vectorized(point1, point2):
    # 计算两点之间的角度
    delta_x = point2[..., 0] - point1[..., 0]
    delta_y = point2[..., 1] - point1[..., 1]
    angle = np.arctan2(delta_y, delta_x) * 180 / np.pi
    return angle
def cal_angle_and_mask_matrix_vectorized(tasks):
    n_epoch, n_batch_per_epoch, batch_size, n_node, _ = tasks.shape
    tasks_reshaped = tasks.reshape(n_epoch, n_batch_per_epoch, batch_size, n_node, 1, 2)
    tasks_broadcasted = np.broadcast_to(tasks_reshaped, (n_epoch, n_batch_per_epoch, batch_size, n_node, n_node, 2))
    tasks_transposed = tasks_broadcasted.transpose(0, 1, 2, 4, 3, 5)
    
    angle = calculate_angle_vectorized(tasks_broadcasted, tasks_transposed)
    mask_matrix = ((angle > -30) & (angle < 30)).astype(int)
    
    return angle, mask_matrix

In [32]:
time_start = time.time()
angle1, mask_matrix1 = cal_angle_and_mask_matrix(tasks)
time_end = time.time()
print('time cost', time_end-time_start, 's')



time cost 0.7272896766662598 s


In [33]:
time_start = time.time()
angle2, mask_matrix2 = cal_angle_and_mask_matrix_vectorized(tasks)
time_end = time.time()
print('time cost', time_end-time_start, 's')

time cost 0.02027297019958496 s


In [35]:
np.array_equal(angle1, angle2)

False

In [37]:
np.where(angle1 != angle2)

(array([0, 0, 0, ..., 1, 1, 1]),
 array([ 0,  0,  0, ..., 99, 99, 99]),
 array([0, 0, 0, ..., 9, 9, 9]),
 array([ 0,  0,  0, ..., 19, 19, 19]),
 array([ 1,  2,  3, ..., 16, 17, 18]))